In [1]:
# Установка необходимых библиотек
!pip install torch torchvision torchaudio gdown
!pip install numpy pandas tqdm opencv-python ffmpeg-python

# Клонирование репозитория
!git clone https://github.com/soumik-kanad/diff2lip.git
%cd diff2lip

# Установка зависимостей
!pip install -r requirements.txt

fatal: destination path 'diff2lip' already exists and is not an empty directory.
/content/diff2lip
Obtaining file:///content/diff2lip/guided-diffusion (from -r requirements.txt (line 11))
  Preparing metadata (setup.py) ... done
  Attempting uninstall: guided-diffusion
    Found existing installation: guided-diffusion 0.0.0
    Uninstalling guided-diffusion-0.0.0:
      Successfully uninstalled guided-diffusion-0.0.0
  Running setup.py develop for guided-diffusion


In [2]:
import torch

!cd diff2lip

!mkdir checkpoints

!cd checkpoints

/bin/bash: line 1: cd: diff2lip: No such file or directory
mkdir: cannot create directory ‘checkpoints’: File exists


In [3]:
import os
import gdown

# URL файла на Google Drive
url = 'https://drive.google.com/uc?id=15vMcyPtFDEttNwelP3IouwsMpnccTqit'

# Имя файла для сохранения
output_path = 'checkpoints/diff2lip_checkpoint.pt'

# Скачивание файла
gdown.download(url, output_path, quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=15vMcyPtFDEttNwelP3IouwsMpnccTqit
From (redirected): https://drive.google.com/uc?id=15vMcyPtFDEttNwelP3IouwsMpnccTqit&confirm=t&uuid=1d73a433-7505-4e2c-af4b-490edc4b3d6f
To: /content/diff2lip/checkpoints/diff2lip_checkpoint.pt
100%|██████████| 407M/407M [00:12<00:00, 33.7MB/s]


'checkpoints/diff2lip_checkpoint.pt'

In [4]:
import os
import pickle
import torch

# Пути к файлам
video_path = "video1.mp4"
audio_path = "audio1.mp3"
model_path = "/content/diff2lip/checkpoints/diff2lip_checkpoint.pt"
script_path = "scripts/inference.sh"
# Создание скрипта inference_single_video.sh
with open(script_path, 'w') as f:
    f.write(f'''#!/bin/bash

#set paths and arguments
sample_mode="cross" # or "reconstruction"
NUM_GPUS=1
generate_from_filelist=0
video_path="video1.mp4"
audio_path="audio1.mp3"
out_path="output.mp4"
model_path="checkpoints/diff2lip_checkpoint.pt"



#cross vs reconstruction
if [ "$sample_mode" = "reconstruction" ]; then
    sample_input_flags="--sampling_input_type=first_frame --sampling_ref_type=first_frame"
elif [ "$sample_mode" = "cross" ]; then
    sample_input_flags="--sampling_input_type=gt --sampling_ref_type=gt"
else
    echo "Error: sample_mode can only be \"cross\" or \"reconstruction\""
    exit 0
fi

MODEL_FLAGS="--attention_resolutions 32,16,8 --class_cond False --learn_sigma True --num_channels 128 --num_head_channels 64 --num_res_blocks 2 --resblock_updown True --use_fp16 True --use_scale_shift_norm False"
DIFFUSION_FLAGS="--predict_xstart False  --diffusion_steps 1000 --noise_schedule linear --rescale_timesteps False"
SAMPLE_FLAGS="--sampling_seed=7   $sample_input_flags --timestep_respacing ddim25 --use_ddim True --model_path=$model_path"
DATA_FLAGS="--nframes 5 --nrefer 1 --image_size 128 --sampling_batch_size=32 "
TFG_FLAGS="--face_hide_percentage 0.5 --use_ref=True --use_audio=True --audio_as_style=True"
GEN_FLAGS="--generate_from_filelist $generate_from_filelist  --video_path=$video_path --audio_path=$audio_path --out_path=$out_path --save_orig=False --face_det_batch_size 64 --pads 0,0,0,0 --is_voxceleb2=False"


NEW_FLAGS="--generate_from_filelist=0 --video_path=video1.mp4 --audio_path=audio1.mp3 --out_path=output.mp4"

if  [ "$NUM_GPUS" -gt 1 ]; then
    mpiexec -n $NUM_GPUS python generate_dist.py $MODEL_FLAGS  $DIFFUSION_FLAGS  $SAMPLE_FLAGS $DATA_FLAGS $TFG_FLAGS $GEN_FLAGS
else
    python generate.py $MODEL_FLAGS  $DIFFUSION_FLAGS  $SAMPLE_FLAGS $DATA_FLAGS $TFG_FLAGS $NEW_FLAGS
fi
''')

# Запуск скрипта
!bash {script_path}


2024-05-24 08:34:33.706340: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-24 08:34:33.706404: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-24 08:34:33.708560: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-24 08:34:33.719541: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-24 08:34:35.554139: W tensorflow/compiler/tf2

In [5]:
import tensorflow as tf
from tensorflow.python.framework import ops

# Удалите все зарегистрированные фабрики
ops.reset_default_graph()
